# Used Car Regression
**ML Project 1 - Spring 2022**

Collaborators: Ben DeSollar and Matt McDonell

Through research we found that a called GradientBoostingRegressor worked the best after testing a couple different model including
    KNeighborsRegressor, Lasso, Ridge, and DecisionTreeRegressor. We improved our results by modifying our feature selection to include as many features as we 
    could to help predict the best values. Things like weight and Deaft year did not seem to be as important as the draft round and PIE. By modifying our featuiues 
    during testing we were able to find the best model predictions.

### Step 1
Import the required packages and frameworks.

In [69]:
# imports
import numpy as np
from mlwpy import *
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import seaborn as sns
from sklearn import (datasets, neighbors,
                     naive_bayes,
                     model_selection as skms,
                     linear_model, dummy,
                     metrics,
                     pipeline,
                     preprocessing as skpre) 
import csv
from sklearn import tree
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor

import numpy as np
import pandas as pd
from sklearn import datasets
import seaborn as sns
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from category_encoders.ordinal import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce

### Step 2
Read in the .csv files and create DataFrames

In [70]:
data_train_df = pd.read_csv("cars_reg_train.csv") 
data_test_df = pd.read_csv("cars_reg_test.csv")
data_train_ft = data_train_df.drop('price', axis=1)
data_train_tgt = data_train_df["price"]

### Step 3
Focus down to the features we think will have the largest impact on points scored. 

We tried to stick to more of the physical attributes like age or height over subjective factors, like city or roster status. We believe points are more about the players themselves than the citys/schools they play for.
As well as including whether they are in the NBA or not, active, and the draft selection. Lastly PIE seems to have the greatest impact, even though there are not a lot of them in the data. 

In [71]:
# To see what kind of data we're working with
print(data_train_ft.dtypes)
print(data_train_ft.head(5))

# From looking at the data it looks like Draft and all star apperances should be useful. 
# Feauture used will need to be transformed into numbers in order to use the prediction models and have them be uniform≥ 

Unnamed: 0        int64
url              object
region           object
region_url       object
year            float64
manufacturer     object
model            object
condition        object
cylinders        object
fuel             object
odometer        float64
title_status     object
transmission     object
VIN              object
drive            object
size             object
type             object
paint_color      object
image_url        object
description      object
state            object
lat             float64
long            float64
posting_date     object
dtype: object
   Unnamed: 0                                                url  \
0      259073  https://newjersey.craigslist.org/cto/d/bloomin...   
1      445077  https://denver.craigslist.org/cto/d/henderson-...   
2      109862  https://spacecoast.craigslist.org/cto/d/merrit...   
3       17375  https://yuma.craigslist.org/cto/d/yuma-2016-fo...   
4       51906  https://sacramento.craigslist.org/ctd/d/san-ra...   

 

### Step 4
Convert numbers to their correct formats and fill in any missing values. 

Missing values don't help us in our quest for accuracy, but filling them in as zero won't necessarily hurt us either. By filling them in, we can still use other parts of the player's data without having to throw away the entire entry.

In [72]:

# 5 6 8 12 13

features = [#'url',
           #'region',
           #'region_url',
           'year',
           'manufacturer',
           'model',
           'condition',
           #'cylinders',
           'fuel',
           'odometer',
           #'title_status',
           'transmission',
           #'VIN',
           'drive',
           'size',
           'type',
           'paint_color',
           #'image_url',
           #'description',
           'state',
           #'lat',
           #'long',
          # 'posting_date'
          ]
data_train_ft['posting_date'] = pd.to_datetime(data_train_ft['posting_date'], utc=True)
data_train_ft['posting_date'] = pd.to_datetime(data_train_ft['posting_date'])
data_train_ft['posting_date'] = (pd.DatetimeIndex(data_train_ft['posting_date']).day) + (pd.DatetimeIndex(data_train_ft['posting_date']).month*12) + ((2022 - pd.DatetimeIndex(data_train_ft['posting_date']).year)*365)
data_train_ft['posting_date'] = data_train_ft['posting_date'].astype(float)
#print(data_train_ft['posting_date'].head(5))
data_test_df['posting_date'] = pd.to_datetime(data_test_df['posting_date'], utc=True)
data_test_df['posting_date'] = pd.to_datetime(data_test_df['posting_date'])
data_test_df['posting_date'] = (pd.DatetimeIndex(data_test_df['posting_date']).day) + (pd.DatetimeIndex(data_test_df['posting_date']).month*12) + ((2022 - pd.DatetimeIndex(data_test_df['posting_date']).year)*365)
data_test_df['posting_date'] = data_test_df['posting_date'].astype(float)
#print(data_test_df['posting_date'].head(5))

data_train_ft = data_train_ft[features]
data_train_ft = data_train_ft.fillna(0)

data_test_ft = data_test_df
data_test_df = data_test_df[features]
data_test_df = data_test_df.fillna(0)

'''
TE_encoder = TargetEncoder()
data_train_ft = TE_encoder.fit_transform(data_train_ft, data_train_tgt)
data_test_df = TE_encoder.transform(data_test_df)
'''
'''
LOOE_encoder = LeaveOneOutEncoder()
data_train_ft = LOOE_encoder.fit_transform(data_train_ft, data_train_tgt)
data_test_df = LOOE_encoder.transform(data_test_df)
'''
encoder = ce.BinaryEncoder()
data_train_ft = encoder.fit_transform(data_train_ft)
data_test_df = encoder.fit_transform(data_test_df)
print(data_train_ft.head(10))

#LE_encoder = OrdinalEncoder(features)
#data_train_ft = LE_encoder.fit_transform(data_train_ft)
#data_test_df = LE_encoder.transform(data_test_df)
#data_train_ft.head()

'''
ftrsel = ftr_sel.SelectKBest(ftr_sel.f_classif, k=5)
ftrsel.fit_transform(data_train_ft, data_train_tgt)

keepers_idx = ftrsel.get_support()
print(keepers_idx)
# use target encoding to encode two categorical features

ftrsel = ftr_sel.SelectFromModel(ensemble.RandomForestClassifier(), 
                                 threshold='mean') # default
'''
#ftrsel = ftr_sel.RFE(linear_model.LinearRegression(),
                   #  n_features_to_select=5)
'''
train_plus_validation_ftrs, test_ftrs, train_plus_validation_tgt, test_tgt = train_test_split (data_train_ft, data_train_tgt, test_size = 0.20,
                                   random_state = 42)
# ordinal encode input variables
ordinal_encoder = OrdinalEncoder(features)
ordinal_encoder.fit(train_plus_validation_ftrs)
train_plus_validation_ftrs = ordinal_encoder.transform(train_plus_validation_ftrs)
test_ftrs = ordinal_encoder.transform(test_ftrs)
print(train_plus_validation_ftrs.head())

test_ftrs = test_ftrs.astype(float)
train_plus_validation_ftrs = train_plus_validation_ftrs.astype(float)

# ordinal encode target variable
encoder = ce.BinaryEncoder()
train_plus_validation_tgt = encoder.fit_transform(train_plus_validation_tgt)
test_tgt = encoder.fit_transform(test_tgt)
print(train_plus_validation_tgt.head())

train_plus_validation_tgt = train_plus_validation_tgt.astype(float)
test_tgt = test_tgt.astype(float)
'''

lmlr = linear_model.LogisticRegression
ftrsel = ftr_sel.SelectFromModel(lmlr(penalty='l2')) # thesh is "small" coeffs
#ftrsel = ftr_sel.RFE(ensemble.RandomForestClassifier(),
                   #n_features_to_select=7)
ftrsel.fit_transform(data_train_ft, data_train_tgt)

print(np.array(data_train_ft.columns.value_counts().index)[ftrsel.get_support()])
print(data_train_ft.columns.value_counts().count())
#ftrsel = ftr_sel.RFE(ensemble.RandomForestClassifier(),
                    # n_features_to_select=7)
new_features = np.array(data_train_ft.columns.value_counts().index)[ftrsel.get_support()]
data_train_ft = data_train_ft[new_features]
data_test_df = data_test_df[new_features]

                  year  manufacturer_0  manufacturer_1  manufacturer_2  \
0           2,010.0000               0               0               0   
1           1,967.0000               0               0               0   
2           1,999.0000               0               0               0   
3           2,016.0000               0               0               0   
4           2,017.0000               0               0               0   
5           2,010.0000               0               0               0   
6           2,017.0000               0               0               0   
7           2,008.0000               0               0               0   
8           2,001.0000               0               0               0   
9           2,018.0000               0               0               0   

   manufacturer_3  manufacturer_4  manufacturer_5  model_0  model_1  model_2  \
0               0               0               1        0        0        0   
1               0        

Show the updated Dataframes

In [ ]:

print("data_train_ft:")
display(data_train_ft.head(10))
print("data_train_ft.info():")
display(data_train_ft.info())

data_train_ft = data_train_ft.astype(float)
data_test_df = data_test_df.astype(float)

print("data_train_ft:")
display(data_train_ft.head(10))
print("data_train_ft.info():")
display(data_train_ft.info())


data_train_ft:


,price
139008,"8,995.0000"
150724,"34,995.0000"
63979,"5,200.0000"
182137,"11,900.0000"
73464,"12,400.0000"
51249,"3,500.0000"
85241,"6,950.0000"
194333,"46,995.0000"
18596,"24,000.0000"
173265,"9,995.0000"


data_train_ft.info():
<class 'pandas.core.frame.DataFrame'>
Int64Index: 160732 entries, 139008 to 121958
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   price   160732 non-null  float64
dtypes: float64(1)
memory usage: 2.5 MB


None

data_train_ft:


,price
139008,"8,995.0000"
150724,"34,995.0000"
63979,"5,200.0000"
182137,"11,900.0000"
73464,"12,400.0000"
51249,"3,500.0000"
85241,"6,950.0000"
194333,"46,995.0000"
18596,"24,000.0000"
173265,"9,995.0000"


data_train_ft.info():
<class 'pandas.core.frame.DataFrame'>
Int64Index: 160732 entries, 139008 to 121958
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   price   160732 non-null  float64
dtypes: float64(1)
memory usage: 2.5 MB


None

### Step 5
Create a heat map to visualize trends present in the data.

In [ ]:
correlation_map = np.corrcoef(data_train_ft.values.T)
sns.set(font_scale=1.0)
heatmap = sns.heatmap(correlation_map, cbar=True, annot=True, square=True, fmt='.2f', yticklabels=features, xticklabels=features)

plt.show()

NameError: name 'new_features' is not defined

### Step 6
Split the data into training and validation sets for use with cross-validation scoring. We split the data randomly to lower the chances of overfitting.

In [ ]:
train_plus_validation_ftrs, test_ftrs, train_plus_validation_tgt, test_tgt = train_test_split (data_train_ft, data_train_tgt, test_size = 0.20,
                                   random_state = 42)

### Step 7
This is another portion where accuracy is most directly influenced. We added as many models as possible (within reason) to ensure that we were covering all our bases to get the most accurate predictions.

In [ ]:

pipelines = {
    'LR' : LinearRegression(),
    'EL Net' : ElasticNet(random_state=0),
    #'Multi-Net' : linear_model.MultiTaskElasticNet(alpha=0.1)
    #'CART' : DecisionTreeRegressor(max_depth=10)
    }
for k in range(50,120,30):
    pipelines[f'RandomForest-{k}'] = sklearn.ensemble.RandomForestRegressor(n_estimators=k)
#for k in range(1,11,2):
    #pipelines[f'KNN-{k}'] = KNeighborsRegressor(n_neighbors=k)
#for i in range(25,45,5):
    #pipelines[f'GBR-{i}'] = GradientBoostingRegressor(random_state=21, n_estimators=i)
values = [40, 50, 100]
for alpha_value in values:
    pipelines[f'Lasso (C={alpha_value})'] = linear_model.Lasso(alpha=alpha_value)
    pipelines[f'Ridge (C={alpha_value})'] = linear_model.Ridge(alpha=alpha_value)
print(pipelines)




{'LR': LinearRegression(), 'EL Net': ElasticNet(random_state=0), 'RandomForest-50': RandomForestRegressor(n_estimators=50), 'RandomForest-80': RandomForestRegressor(n_estimators=80), 'RandomForest-110': RandomForestRegressor(n_estimators=110), 'Lasso (C=40)': Lasso(alpha=40), 'Ridge (C=40)': Ridge(alpha=40), 'Lasso (C=50)': Lasso(alpha=50), 'Ridge (C=50)': Ridge(alpha=50), 'Lasso (C=100)': Lasso(alpha=100), 'Ridge (C=100)': Ridge(alpha=100)}


### Step 8
Using different standardizing pipelines, run multiple cross-validation tests on the data splits to find the best performing model.

Further refine the selection of best model by comparing scores.

In [ ]:

accuracy_scores = {}
for pipeline_name in pipelines:
    #loo = skms.LeaveOneOut()
    print(f'Currently testing model: {pipeline_name}')
    scores = skms.cross_val_score(pipelines[pipeline_name],
                                  train_plus_validation_ftrs,
                                  train_plus_validation_tgt,
                                  #cv=loo,
                                  cv=10,
                                  scoring='neg_mean_squared_error')
    scores = np.sqrt(-scores.mean())
    accuracy_scores[pipeline_name] = scores
    print(f'{pipeline_name}: {scores:.3f}')

best_model_name = min(accuracy_scores,key=accuracy_scores.get)
print(f'\nBest model: {best_model_name} (accuracy = {accuracy_scores[best_model_name]:.3f})')
final_model = pipelines[best_model_name]

## Took my computer 3 min 14 sec to run this

Currently testing model: LR


ValueError: Found input variables with inconsistent numbers of samples: [160732, 200916]

### Step 9
Rescale the data and fit the model.

In [ ]:
scaler = StandardScaler().fit(train_plus_validation_ftrs)
rescaled_train_plus_validation_ftrs = scaler.transform(train_plus_validation_ftrs)
car_scores_fit = final_model.fit(rescaled_train_plus_validation_ftrs, 
                                   train_plus_validation_tgt)
rescaled_test_ftrs = scaler.transform(test_ftrs)

### Step 10
Scale the data and predict points on the selected model and save it in a temporary .csv for analysis.

In [ ]:

predictions = car_scores_fit.predict(rescaled_test_ftrs)
# score = np.sqrt(metrics.mean_squared_error(predictions, tes))
print(f'Using model: {final_model} and features: {new_features}')
# print(f'Final score: {score:.3f}')
# predictions = model.predict(rescaled_test_ftrs)

# compare = pd.DataFrame({'Prediction': predictions, 'Test Data' : test_tgt})
# compare.head(10)
# This is just using the test.csv to setup a dataframe of the correct size
# and indicies (the "id" field).
make_submission_df = pd.read_csv("cars_reg_test.csv")
# drop all columns except 'id'
make_submission_df = make_submission_df[['id']]
# make sure the column of ID's that we just read in is the index column
make_submission_df = make_submission_df.set_index('id')

Using model: ElasticNet(random_state=0) and features: ['fuel' 'title_status' 'drive' 'size' 'type']


### Step 11
Add our predictions to a submission file and save the final .csv

In [ ]:
make_submission_df['price'] = predictions
make_submission_df.to_csv('submission.csv',sep=',', float_format='%.2f')